In [1]:
!pip install pydub

In [2]:
!pip install librosa

In [3]:
!pip install SoundFile

In [4]:
!pip install fastdtw

In [5]:
import os
import numpy as np
import librosa
import soundfile as sf

import json

import shutil

from fastdtw import fastdtw

import glob


# E_Get_Cuts

In [6]:
gray_values_list = [0.11594238132238388, 0.3915926218032837, 0.17946207523345947, 0.11579568684101105, 0.20035278797149658, 0.435150682926178, 0.1846507489681244, 0.10584469884634018, 0.0714426040649414, 0.4386752247810364, 0.7747208476066589, 0.7569605708122253, 0.8806381821632385, 0.7452989816665649, 0.8491743803024292, 0.8340145349502563, 0.8301460146903992, 0.8449164032936096, 0.612399160861969, 0.3856951594352722, 0.16029906272888184, 0.056268900632858276, 0.06105627864599228, 0.012285354547202587, 0.07373790442943573, 0.025469280779361725, 0.009900880977511406, 0.07399658113718033, 0.31513744592666626, 0.08416632562875748, 0.09452623128890991, 0.22724978625774384, 0.1494787335395813, 0.06788448989391327, 0.18561811745166779, 0.1157396137714386, 0.21956488490104675, 0.12330926209688187, 0.1472208946943283, 0.054550476372241974, 0.06711207330226898, 0.042803164571523666, 0.06836268305778503, 0.11239665001630783, 0.07547735422849655, 0.006306773517280817, 0.005367566365748644, 0.0209212526679039, 0.3068385124206543, 0.6169638633728027, 0.19812457263469696, 0.24464070796966553, 0.4563194215297699, 0.7609230279922485, 0.6695041656494141, 0.7427549362182617, 0.6097912192344666, 0.24915441870689392, 0.3276250958442688, 0.21614763140678406, 0.13737823069095612, 0.06740164756774902, 0.11052271723747253, 0.29452958703041077, 0.4031125009059906, 0.19935031235218048, 0.1064632385969162, 0.16320852935314178, 0.17940638959407806, 0.17115944623947144, 0.2269391119480133, 0.10786835849285126, 0.07634145766496658, 0.19346560537815094, 0.06558264791965485, 0.05629140883684158, 0.06454043090343475, 0.09128515422344208, 0.07176409661769867, 0.009945180267095566, 0.00992581807076931, 0.0143069913610816, 0.023299476131796837, 0.012910336256027222, 0.02659565582871437, 0.03556235879659653, 0.02303113043308258, 0.014162944629788399, 0.042469047009944916, 0.020663822069764137, 0.03078395128250122, 0.03882342204451561]

In [7]:
def get_sample_rate(audio_file):
    _, sample_rate = librosa.load(audio_file, sr=None)
    return sample_rate

# Replace with the path to your audio file
audio_file = 'Istanbul alley6.wav'
sample_rate = get_sample_rate(audio_file)

print(f"Sample rate of the audio file: {sample_rate} Hz")
def crop_audio_by_grayscale(wav_file, grayscale_data, output_folder_cuts, threshold=0.1, min_duration=0.1, sample_rate=sample_rate, end_min_duration=2.0):
    # Load the audio file
    audio_data, sr = librosa.load(wav_file, sr=sample_rate)

    # Calculate the time step (frame rate)
    time_step = len(audio_data) / len(grayscale_data)

    # Find the start and end points of the sound clips
    clip_start = -1
    clip_end = -1
    clips = []

    for i, value in enumerate(grayscale_data):
        if value > threshold and clip_start == -1:
            clip_start = i
        elif value < threshold and clip_start != -1:
            clip_end = i

            # Check if the duration is greater than the end minimum duration
            duration = (clip_end - clip_start) * time_step / sample_rate
            if duration >= end_min_duration:
                clips.append((clip_start, clip_end))
                clip_start = -1

    print(f"Number of clips found: {len(clips)}")

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder_cuts, exist_ok=True)

    # Crop and save the sound clips
    for idx, (start, end) in enumerate(clips):
        start_time = start * time_step / sample_rate
        end_time = end * time_step / sample_rate
        duration = end_time - start_time

        # Check if the duration is greater than the minimum duration
        if duration >= min_duration:
            cropped_audio = audio_data[int(start * time_step):int(end * time_step)]
            output_file = os.path.join(output_folder_cuts, f"clip_{idx+1}_{start_time:.2f}_{duration:.2f}.wav")
            sf.write(output_file, cropped_audio, sample_rate)

            print(f"Saved clip: {output_file}")
        else:
            print(f"Clip {idx+1} with duration {duration:.2f}s is shorter than the minimum duration, so it was not saved.")

wav_file_name = audio_file
grayscale_data = gray_values_list
output_folder_cuts = 'output_folder_cuts'

crop_audio_by_grayscale(wav_file_name, grayscale_data, output_folder_cuts)

        

Sample rate of the audio file: 48000 Hz
Number of clips found: 3
Saved clip: output_folder_cuts\clip_1_0.00_2.29.wav
Saved clip: output_folder_cuts\clip_2_3.06_2.07.wav
Saved clip: output_folder_cuts\clip_3_5.24_2.62.wav


# F_audio classification

In [8]:
def searchForMatch(input_audio_name, input_feature_dict, library_feature_dict, key_segment_start, key_segment_stop, sr, nBestMatches=1, hop_length=512):
    input_features = input_feature_dict[input_audio_name][:, key_segment_start:key_segment_stop]
    best_matches = []

    for library_audio_name, library_features in library_feature_dict.items():
        min_dtw_distance = float('inf')
        min_dtw_start = None
        min_dtw_end = None

        for i in range(library_features.shape[1] - input_features.shape[1] + 1):
            segment_features = library_features[:, i:i + input_features.shape[1]]
            distance, _ = fastdtw(input_features.T, segment_features.T)
            if distance < min_dtw_distance:
                min_dtw_distance = distance
                min_dtw_start = i
                min_dtw_end = i + input_features.shape[1]

        best_matches.append((library_audio_name, min_dtw_start, min_dtw_end, min_dtw_distance))

    best_matches.sort(key=lambda x: x[3])
    best_match = best_matches[:nBestMatches][0]

    return best_match[0], best_match[1] * hop_length / sr, best_match[2] * hop_length / sr


In [9]:
def main(input_folder, library_folder, output_folder, original_audio, sr=sample_rate, n_mels=128, hop_length=512):
    # Load input audio files
    input_audio_files = os.listdir(input_folder)
    input_feature_dict = {}
    for af in input_audio_files:
        if af.split('.')[-1] in ['mp3', 'wav']:
            logMelSpec = extractFeatures(os.path.join(input_folder, af), sr=sr, n_mels=n_mels, hop_length=hop_length)
            input_feature_dict[af] = logMelSpec

    # Load library audio files
    library_audio_files = os.listdir(library_folder)
    library_feature_dict = {}
    for af in library_audio_files:
        if af.split('.')[-1] in ['mp3', 'wav']:
            logMelSpec = extractFeatures(os.path.join(library_folder, af), sr=sr, n_mels=n_mels, hop_length=hop_length)
            library_feature_dict[af] = logMelSpec

    # Compare input audio segments to library audio files
    for input_audio_name, input_features in input_feature_dict.items():
        key_segment_start = 0
        key_segment_stop = input_features.shape[1]

        match_audio_name, match_start, match_end = searchForMatch(
            input_audio_name,
            input_feature_dict,  # Use input_feature_dict instead of library_feature_dict for input audio
            library_feature_dict,
            key_segment_start,
            key_segment_stop,
            sr,
            nBestMatches=1,
            hop_length=hop_length,
        )

        print(f"Input audio file: {input_audio_name}")
        print(f"Most similar segment found in {match_audio_name}: {match_start:.2f}s - {match_end:.2f}s")

        # Export matched audio segment
        match_audio_path = os.path.join(library_folder, match_audio_name)
        output_audio_name = f"{input_audio_name[:-4]}_{match_audio_name[:-4]}_{match_start:.2f}.wav"
        output_audio_path = os.path.join(output_folder, output_audio_name)
        export_segment(match_audio_path, output_audio_path, match_start, match_end, sr)
        
# Concatenate matched audio segments according to the start times of the original audio clips
    original_audio_data, _ = librosa.load(original_audio, sr=sr)
    final_audio_data = np.zeros_like(original_audio_data)

    for input_audio_name in input_audio_files:
        if input_audio_name.split('.')[-1] in ['mp3', 'wav']:
            start_time = float(input_audio_name.split('_')[-2])
            input_audio_path = os.path.join(input_folder, input_audio_name)
            input_audio_data, _ = librosa.load(input_audio_path, sr=sr)
            input_audio_length = len(input_audio_data)

            match_audio_name = f"{input_audio_name[:-4]}_*.wav"
            match_audio_path = glob.glob(os.path.join(output_folder, match_audio_name))[0]
            match_audio_data, _ = librosa.load(match_audio_path, sr=sr)

            start_sample = int(start_time * sr)
            final_audio_data[start_sample:start_sample + input_audio_length] = match_audio_data[:input_audio_length]

    # Save the final concatenated audio
    output_audio_path = os.path.join(output_folder, 'final_concatenated_audio.wav')
    sf.write(output_audio_path, final_audio_data, sr)

In [10]:
def extractFeatures(audioFile, sr, n_mels=128, hop_length=512):
    y, sr = librosa.load(audioFile, sr=sr)
    melSpec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)

    logMelSpec = librosa.power_to_db(melSpec, ref=np.max)
    return logMelSpec

def export_segment(audio_path, output_path, start_time, end_time, sr):
    y, sr = librosa.load(audio_path, sr=sr)
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    y_segment = y[start_sample:end_sample]
    sf.write(output_path, y_segment, sr)


In [14]:
original_audio = audio_file
input_folder = output_folder_cuts
library_folder = "audioexamples"
output_folder = "output_folder_E"
main(input_folder, library_folder, output_folder, original_audio)


Input audio file: clip_1_0.00_2.29.wav
Most similar segment found in 07-carnatic.wav: 2.78s - 5.08s
Input audio file: clip_2_3.06_2.07.wav
Most similar segment found in 06-narration.wav: 3.05s - 5.13s
Input audio file: clip_3_5.24_2.62.wav
Most similar segment found in 08-asian.wav: 2.05s - 4.67s
